<a href="https://colab.research.google.com/github/Shibu4064/Thesis_Final/blob/main/data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
nltk.download('stopwords-bn')

[nltk_data] Error loading stopwords-bn: Package 'stopwords-bn' not
[nltk_data]     found in index


False

In [ ]:
!pip install np_utils
!pip install keras

In [ ]:
import pandas as pd
import numpy as np
import random
import warnings
import time
import datetime
import re
import string
import itertools
import pickle
import joblib
import nltk
import csv
import keras.utils
from nltk.corpus import stopwords, wordnet
stop = set('stopwords-bn')
from wordcloud import WordCloud, STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from collections import Counter, defaultdict

import tensorflow as tf
import keras
import keras.backend as K
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Concatenate, Conv2D, Flatten, Dense, Embedding, LSTM
from keras.models import Model
#from keras.utils import np_utils
from tensorflow.keras.utils import to_categorical

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from keras.regularizers import l2
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from tensorflow.keras.layers import BatchNormalization

from keras.layers import MaxPooling2D
from keras.layers import Concatenate
from keras.layers import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from keras.layers import Input, Dense, Flatten, GlobalMaxPool2D, GlobalAvgPool2D, Concatenate, Multiply, Dropout, Subtract, Add, Conv2D

from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_excel('/content/annnotated.xlsx')
#df.drop(['keyword', 'location', 'id'], axis=1, inplace=True)
display(df)

,Headline,Content,Label
0,ছোট্ট বন্ধুর কান ধরে টান দিলেন প্রধানমন্ত্রী ম...,ভারতের প্রধানমন্ত্রী নরেন্দ্র মোদির ৬৮ বছর পূর...,0.0
1,"চার্য নাই যার, সেই আচার্য: ইউনূস | দৈনিক মতিকণ্ঠ",স্কটলেন্ড মতিনিধিবাংলাদেশের নোবেল বিজয়ী অর্থন...,0.0
2,নানা ফাঁদে নারী ও তার প্রতিকার,"মাও সে-তুং বলেছিলেন, ‘নারীরা অর্ধেক আকাশ’। কিন...",0.0
3,"রাষ্ট্রপতি ট্রুম্প এমন এক নেটওয়ার্ক চায়, যে ...",আপনার প্রিয় রাষ্ট্রপতি (আমি) “ এক হিলারিয়াস ...,2.0
4,শোচনীয় হারের পর যা বললেন সরফরাজ,চিরপ্রতিদ্বন্দ্বী ভারতের সাথে সবসময়ই তুমুল উত্...,2.0
...,...,...,...
12898,আত্ম পরিচ গভীর সংকট কসা কাদের সন্তান,নিজস্ব মতিবেদক আত্ম পরিচ গভীর সংকট ভুগছ আন্তর্...,2.0
12899,লিস্ট ‘এ’ ক্রিকেটের রেকর্ড ভাঙলেন ভারতীয় স্পিনার,ভারতের এশিয়া কাপ প্রস্তুতিতে সহায়তা করতে সংযুক...,2.0
12900,মিরপুরই হতে যাচ্ছে পৃথিবীর প্রথম আন্ডারওয়াটার ...,ঢাকার অন্যতম দুর্গম ও দর্শনীয় স্থান হিসেবে মির...,2.0
12901,মৃত চিকিৎসকের রেজিস্ট্রেশন নম্বর ‌ব্যবহার করে ...,মৃত চিকিৎসকের রেজিস্ট্রেশন নম্বর ‌ব্যবহার করে ...,2.0


In [ ]:
df.shape

(12903, 3)

In [ ]:
df['Label'].value_counts()

2.0     7422
1.0     3081
0.0     2397
20.0       1
Name: Label, dtype: int64

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
def remove_url(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'', text)

def remove_emoji(text):
    emoji_pattern = re.compile(
        '['
        u'\U0001F600-\U0001F64F'  # emoticons
        u'\U0001F300-\U0001F5FF'  # symbols & pictographs
        u'\U0001F680-\U0001F6FF'  # transport & map symbols
        u'\U0001F1E0-\U0001F1FF'  # flags (iOS)
        u'\U00002702-\U000027B0'
        u'\U000024C2-\U0001F251'
        ']+',
        flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def remove_html(text):
    html = re.compile(r'<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    return re.sub(html, '', text)

def remove_punct(text):
    table = str.maketrans('', '', string.punctuation)
    return text.translate(table)

df['clean_text'] = df['Headline'].apply(lambda x: remove_url(str(x)))
df['clean_text'] = df['clean_text'].apply(lambda x: remove_emoji(str(x)))
df['clean_text'] = df['clean_text'].apply(lambda x: remove_html(str(x)))
df['clean_text'] = df['clean_text'].apply(lambda x: remove_punct(str(x)))
df['clean_text'] = df['clean_text'].apply(word_tokenize)
df['clean_text'] = df['clean_text'].apply(lambda x: [word.lower() for word in x])
df['clean_text'] = df['clean_text'].apply(lambda x: [word for word in x if word not in stop])
df['clean_text'] = df['clean_text'].apply(nltk.tag.pos_tag)

def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

df['clean_text'] = df['clean_text'].apply(
    lambda x: [(word, get_wordnet_pos(pos_tag)) for (word, pos_tag) in x])
wnl = WordNetLemmatizer()
df['clean_text'] = df['clean_text'].apply(
    lambda x: [wnl.lemmatize(word, tag) for word, tag in x])
df['clean_text'] = df['clean_text'].apply(
    lambda x: [word for word in x if word not in stop])
df['clean_text'] = [' '.join(map(str, l)) for l in df['clean_text']]

In [ ]:
display(df)

,Headline,Content,Label,clean_text
0,ছোট্ট বন্ধুর কান ধরে টান দিলেন প্রধানমন্ত্রী ম...,ভারতের প্রধানমন্ত্রী নরেন্দ্র মোদির ৬৮ বছর পূর...,0.0,ছোট্ট বন্ধুর কান ধরে টান দিলেন প্রধানমন্ত্রী মোদি
1,"চার্য নাই যার, সেই আচার্য: ইউনূস | দৈনিক মতিকণ্ঠ",স্কটলেন্ড মতিনিধিবাংলাদেশের নোবেল বিজয়ী অর্থন...,0.0,চার্য নাই যার সেই আচার্য ইউনূস দৈনিক মতিকণ্ঠ
2,নানা ফাঁদে নারী ও তার প্রতিকার,"মাও সে-তুং বলেছিলেন, ‘নারীরা অর্ধেক আকাশ’। কিন...",0.0,নানা ফাঁদে নারী ও তার প্রতিকার
3,"রাষ্ট্রপতি ট্রুম্প এমন এক নেটওয়ার্ক চায়, যে ...",আপনার প্রিয় রাষ্ট্রপতি (আমি) “ এক হিলারিয়াস ...,2.0,রাষ্ট্রপতি ট্রুম্প এমন এক নেটওয়ার্ক চায় যে ক...
4,শোচনীয় হারের পর যা বললেন সরফরাজ,চিরপ্রতিদ্বন্দ্বী ভারতের সাথে সবসময়ই তুমুল উত্...,2.0,শোচনীয় হারের পর যা বললেন সরফরাজ
...,...,...,...,...
12898,আত্ম পরিচ গভীর সংকট কসা কাদের সন্তান,নিজস্ব মতিবেদক আত্ম পরিচ গভীর সংকট ভুগছ আন্তর্...,2.0,আত্ম পরিচ গভীর সংকট কসা কাদের সন্তান
12899,লিস্ট ‘এ’ ক্রিকেটের রেকর্ড ভাঙলেন ভারতীয় স্পিনার,ভারতের এশিয়া কাপ প্রস্তুতিতে সহায়তা করতে সংযুক...,2.0,লিস্ট ‘ এ ’ ক্রিকেটের রেকর্ড ভাঙলেন ভারতীয় স্প...
12900,মিরপুরই হতে যাচ্ছে পৃথিবীর প্রথম আন্ডারওয়াটার ...,ঢাকার অন্যতম দুর্গম ও দর্শনীয় স্থান হিসেবে মির...,2.0,মিরপুরই হতে যাচ্ছে পৃথিবীর প্রথম আন্ডারওয়াটার ...
12901,মৃত চিকিৎসকের রেজিস্ট্রেশন নম্বর ‌ব্যবহার করে ...,মৃত চিকিৎসকের রেজিস্ট্রেশন নম্বর ‌ব্যবহার করে ...,2.0,মৃত চিকিৎসকের রেজিস্ট্রেশন নম্বর ‌ব্যবহার করে ...
